# Выбор локации для скважины

В датасете собраны пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Необходимо построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Необходимо проанализировать возможную прибыль и риски.

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

**Описание данных**
`id` — уникальный идентификатор скважины  
`f0`, `f1`, `f2` — три признака точек    
`product` — объём запасов в скважине (тыс. баррелей)

## Загрузка и подготовка данных

In [52]:
import os
from math import sqrt
from numpy.random import RandomState

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as mse

In [53]:
BASE_DIR = os.getcwd()
r_state = RandomState(12345)
state = 1

Загружаем датасеты.

In [54]:
d_1 = pd.read_csv(f'{BASE_DIR}/datasets/geo_data_0.csv')
d_2 = pd.read_csv(f'{BASE_DIR}/datasets/geo_data_1.csv')
d_3 = pd.read_csv(f'{BASE_DIR}/datasets/geo_data_2.csv')

Анализ общей информации о данных.

In [55]:
d_1.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [56]:
d_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [57]:
d_1.isna().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

Пропуски в данных отсутствуют.   
Необходимо удалить лишний столбец с идентификаторами данных, т.к. при обучении модели они использоваться не будет, а если нам в дальнейшем понадобятся идентификаторы мы сможем восстановить их по индексам из исходного файла.   
Признаки f0, f1, f2 необходимо отмасштабировать.   
Имеет смысл изменить типы данных на более лёгкие.

### 1.1 Удаление столбца с id скважин

In [58]:
d_1 = d_1.drop('id', axis=1)
d_2 = d_2.drop('id', axis=1)
d_3 = d_3.drop('id', axis=1)

In [59]:
d_1.head(5)

,f0,f1,f2,product
0,0.705745,-0.497823,1.221170,105.280062
1,1.334711,-0.340164,4.365080,73.037750
2,1.022732,0.151990,1.419926,85.265647
3,-0.032172,0.139033,2.978566,168.620776
4,1.988431,0.155413,4.751769,154.036647


### 1.2 Проверка дубликатов

In [60]:
print(d_1.duplicated().sum())
print(d_2.duplicated().sum())
print(d_3.duplicated().sum())

0
0
0


### ~~1.3 Масштабирование признаков~~

Этот шаг будет сделан при обучении модели.

### 1.4 Изменение типов

In [61]:
d_1 = d_1.apply(pd.to_numeric, downcast='float')
d_2 = d_2.apply(pd.to_numeric, downcast='float')
d_3 = d_3.apply(pd.to_numeric, downcast='float')

### 1.5 Выводы

Проверка результатов

In [62]:
d_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   f0       100000 non-null  float32
 1   f1       100000 non-null  float32
 2   f2       100000 non-null  float32
 3   product  100000 non-null  float32
dtypes: float32(4)
memory usage: 1.5 MB


In [63]:
d_1.head()

,f0,f1,f2,product
0,0.705745,-0.497822,1.221170,105.280060
1,1.334711,-0.340164,4.365080,73.037750
2,1.022732,0.151990,1.419926,85.265648
3,-0.032172,0.139033,2.978566,168.620773
4,1.988431,0.155413,4.751769,154.036652




**В результате подготовки данных:**
   - были удалены признаки с излишней информацией; 
   - проверены дубликаты и пропуски;
   - ~~количественные признакми были масштабированы;~~
   - типы данных заменены на более лёгкие.

## 2 Обучение и проверка модели

### 2.1 Обучение модели

Функция для обучения модели и расчёта среднего предсказанного запаса сырья и RMSE модели.

In [64]:
def mean_rmse(d):
    feature = d.drop('product', axis=1)
    target = d['product']

    # Разбиваем данные на обучающую и валидационную выборки в соотношении 3:1
    feature_train, feature_valid, target_train, target_valid = train_test_split(
        feature, target, test_size=0.25, random_state=state)

    # Масштабируем признаки
    scaler = StandardScaler()
    scaler.fit(feature_train)
    feature_train = pd.DataFrame(scaler.transform(feature_train))
    feature_valid = pd.DataFrame(scaler.transform(feature_valid))

    # Обучаем модель
    model = LinearRegression()
    model.fit(feature_train, target_train)

    # Делаем предсказание на валидациоонной выборке
    target_predicted = model.predict(feature_valid)
    target_predicted_df = pd.Series(target_predicted, index=target_valid.index)

    # Печатаем на экране средний предсказанный запас сырья и RMSE модели.
    print(
        f'Средний предсказанный запас сырья = {target_predicted.mean()}, RMSE модели = {sqrt(mse(target_valid, target_predicted))}')

    # Возвращаем Series с истиными данными и с предсказанными, они понадобятся позже
    return {'y_pred': target_predicted_df, 'y_true': target_valid}


In [65]:
print('Результаты для d_1 : ', end="")
res_mean_rmse_1 = mean_rmse(d_1)

print('Результаты для d_2 : ', end="")
res_mean_rmse_2 = mean_rmse(d_2)

print('Результаты для d_3 : ', end="")
res_mean_rmse_3 = mean_rmse(d_3)


Результаты для d_1 : Средний предсказанный запас сырья = 92.49263000488281, RMSE модели = 37.74258774497981
Результаты для d_2 : Средний предсказанный запас сырья = 69.12039947509766, RMSE модели = 0.8943376956413401
Результаты для d_3 : Средний предсказанный запас сырья = 94.95684051513672, RMSE модели = 39.866710602428846


### 2.2 Анализ результатов

На участке 2 самый низкий показатель RMSE, значит что модель сможет предсказывать данные на порядок точнее чем на участках 1 и 3. С другой стороны средний предсказанный запас сырья на этом участке меньше.

## 3 Подготовка к расчёту прибыли

 ### 3.1 Все ключевые значения для расчётов сохраним в отдельных переменных.

In [66]:
PLACE_COUNT = 200
PRUDUCT_PRICE = 450_000
BUDGET = 10_000_000_000

### 3.2 Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины. Сравним полученный объём сырья со средним запасом в каждом регионе. 

Стоимость разработки скважин - 10 млрд.  
Стоимость единицы объёма нефти - 450 тыс.р.  
Мы разрабатываем 200 скважин, расходы на 1 скважину 10млрд/200=50 млн.  
Достаточный объём для безубыточной разработки новой скважины 50млн./450т.=111,(1) единиц объёма.

In [67]:
print(d_1['product'].mean())
print(d_2['product'].mean())
print(d_3['product'].mean())

92.5008316040039
68.83883666992188
94.99874877929688


 ### 3.3  Выводы по этапу подготовки расчёта прибыли.

Необходимый объём для безубыточной разработки одной новой скважины - 111,(1) тысяч баррелей. В каждом из регионов средний запас сырья в одной скажине недостаточен. Особенно в регионе d_2. Следовательно, вероятность получить прибыль выбрав места для скважин случайным образом невелика, необходимо использовать машинное обучение.

## 4 Расчёт прибыли и рисков 

### 4.1 Функция для рассчёта прибыли по выбранным скважинам и предсказаниям модели

Случайным образом выбираем 500 скажин из 25000 (для нашей задачи не обязательно брать значения из всей выборки, можно взять из валидационной). Модель уже была обучена ранее, выбираем 500 истиных значений и 500 предсказаных.

In [68]:
# передаём функции истиные значения, предсказанные значения и radnom seed
def revenue(y_true, y_pred, r_stat):

    # выбираем 500 индексов случайных объектов, далее по индексам выбираем сами объекты
    random_500_index = y_true.sample(n=500, random_state=r_stat).index
    y_true_500 = y_true[random_500_index]
    y_pred_500 = y_pred[random_500_index]

    # выбираем 200 лучших объектов из 500 случайных
    indexes_pred_volumes_200 = y_pred_500.sort_values(ascending=False)[
        :PLACE_COUNT].index
    indexes_pred_volumes_200

    # рассчитываем суммарный доход по 200 скажинам
    product_volume_200 = sum(y_true_500[indexes_pred_volumes_200])

    # возвращаем валовую прибыль
    return product_volume_200 * PRUDUCT_PRICE - BUDGET


Функция, которая применяет bootstrap с 1000 выборок, чтобы найти распределение прибыли и находит среднюю прибыль, 95%-й доверительный интервал и риск убытков.

In [69]:
def revenue_bootstrap(*args):
    revenue_list = [revenue(*args, x) for x in range(1000)]
    revenue_series = pd.Series(revenue_list)
    print(f'Доверительный интервал = {revenue_series.quantile(0.025)} - {revenue_series.quantile(0.975)}')
    print(f'Средняя прибыль        = {revenue_series.mean()}')
    print(f'Риск убытков           = {round(100 - (len(revenue_series.loc[revenue_series >= 0])/1000)*100,2)} %')
    return

In [70]:
revenue_bootstrap(res_mean_rmse_1['y_true'], res_mean_rmse_1['y_pred'])

Доверительный интервал = -102060258.08811188 - 923950109.272003
Средняя прибыль        = 425660296.4588938
Риск убытков           = 5.8 %


In [71]:
revenue_bootstrap(res_mean_rmse_2['y_true'], res_mean_rmse_2['y_pred'])

Доверительный интервал = 91717732.2769165 - 875081112.5469208
Средняя прибыль        = 477779668.70803833
Риск убытков           = 0.9 %


In [72]:
revenue_bootstrap(res_mean_rmse_3['y_true'], res_mean_rmse_3['y_pred'])

Доверительный интервал = -145932964.3434286 - 869402555.0526003
Средняя прибыль        = 384331351.84384066
Риск убытков           = 8.0 %


### 4.2 Выводы: предложение региона для разработки скважин и обоснование выбора.  

Лучшим выбором из трёх регионов является второй регион (данные по которому хранятся в файле "geo_data_1.csv").  
В этом регионе самый низкий риск убытков 0.9% против 5,8% и 8% в первом и третьем регионах соответственно, причём показатель средней прибыли на выбранных 200 скажинах выше, чем на других участках.  
По всем значимым метрикам второй регион более предпочтительный, чем два других

## 5 Выводы

В процессе работы были проделаны следующие шаги: 
Данные были подготовлены к работе, удалены лишние столбцы, признаки отмасштабированы, изменены типы данных на более лёгкие.  
Была обучена модель линейной регрессии, а также проанализированы результаты её работы.  
Был рассчитан необходимый запас сырья в скважине для её безубыточной разработки.  
Был проведён рассчёт средней прибыли и рисков уытков с помощью метода bootstrap.  
В результате проведённой работы был рекомендован регион для разработки скважин и приведены обоснования рекомендаций.